## Imbalance Data Analysis

In [1]:
import torch
print(torch.__version__)
print(torch.cuda.is_available())
device=torch.device('cuda',1)
from torch.utils.data import Dataset,DataLoader
from torchvision import transforms,models
import torch.nn as nn
import torch.optim as optim
import copy
import os
from PIL import Image
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from dataloader import getData

1.4.0
True


In [2]:
X,y=getData('train')

In [3]:
count=[]
a=0
for i in range(5):
    count.append(np.count_nonzero(y==i))
    a=max(a,count[-1])
print(count)

for i in range(5):
    print(a/count[i])
    
print(count[0]/sum(count))

[20655, 1955, 4210, 698, 581]
1.0
10.565217391304348
4.906175771971497
29.591690544412607
35.55077452667814
0.73507954019716


## Normalize analysis

In [4]:
class RetinopathyDataSet(Dataset):
    def __init__(self, img_path, mode):
        """
        Args:
            img_path: Root path of the dataset.
            mode: training/testing
            
            self.img_names (string list): String list that store all image names.
            self.labels (int or float list): Numerical list that store all ground truth label values.
        """
        self.img_path = img_path
        self.mode = mode
        
        self.img_names=np.squeeze(pd.read_csv('train_img.csv' if mode=='train' else 'test_img.csv').values)
        self.labels=np.squeeze(pd.read_csv('train_label.csv' if mode=='train' else 'test_label.csv').values)
        assert len(self.img_names)==len(self.labels),'length not the same'
        self.data_len=len(self.img_names)
        
        self.transformations=transforms.Compose([transforms.RandomHorizontalFlip(),transforms.RandomVerticalFlip(),transforms.ToTensor()])
        print(f'>> Found {self.data_len} images...')
        
    def __len__(self):
        return self.data_len

    def __getitem__(self, index):
        single_img_name=os.path.join(self.img_path,self.img_names[index]+'.jpeg')
        single_img=Image.open(single_img_name)  # read an PIL image
        img=self.transformations(single_img)
        label=self.labels[index]
        
        return img, label

In [5]:
dataset_train=RetinopathyDataSet(img_path='data',mode='train')
loader_train=DataLoader(dataset=dataset_train,batch_size=16,shuffle=True,num_workers=4)

>> Found 28099 images...


In [6]:
def compute_mean_std(loader):
    mean = 0.
    std = 0.
    for images, _ in loader:
        batch_samples = images.size(0) # batch size (the last batch can have smaller size!)
        images = images.view(batch_samples, images.size(1), -1)
        mean += images.mean(2).sum(0)
        std += images.std(2).sum(0)

    mean /= len(loader.dataset)
    std /= len(loader.dataset)
    return mean, std

In [7]:
mean,std=compute_mean_std(loader_train)

In [8]:
print(mean)
print(std)

tensor([0.3749, 0.2602, 0.1857])
tensor([0.2526, 0.1780, 0.1291])
